# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [ ]:
#!pip install requests beautifulsoup4

In [2]:
url = "https://books.toscrape.com/.html"

r = requests.get(url)
soup = BeautifulSoup(r.content)
books = soup.find_all("li", attrs = {"class": "col-xs-6 col-sm-4 col-md-3 col-lg-3"})

In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_books(min_rating, max_price, num_books):
    base_url = 'https://books.toscrape.com/catalogue/page-{}.html'
    all_books = []
    
    # Iterate through pages (adjust the range as needed)
    page_number = 1
    while len(all_books) < num_books:
        url = base_url.format(page_number)
        r = requests.get(url)
        
        if r.status_code != 200:
            break  # Stop if there are no more pages (or on error)
        
        soup = BeautifulSoup(r.content, 'html.parser')
        
        books = soup.find_all("li", class_="col-xs-6 col-sm-4 col-md-3 col-lg-3")
        
        for book in books:
            if len(all_books) >= num_books:
                break  # Stop scraping once we've reached the desired number of books

            title = book.find("h3").get_text()
            price = float(book.find("p", class_="price_color").get_text()[1:].strip())
            rating_class = book.find("p", class_="star-rating")['class'][1]
            rating_mapping = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
            rating = rating_mapping.get(rating_class, 0)
            availability = book.find("p", class_="instock availability").get_text().strip()
            link = f"https://books.toscrape.com/catalogue{book.find('h3').find('a')['href'][8:]}"

            # Scrape book page for additional information
            book_page = requests.get(link)
            book_soup = BeautifulSoup(book_page.content, 'html.parser')
            
            # UPC (updated to use string instead of text, and with error handling)
            upc_tag = book_soup.find("th", string="UPC")
            upc = upc_tag.find_next("td").get_text() if upc_tag else "Not Available"
            
            # Genre extraction from the breadcrumb navigation
            breadcrumb = book_soup.find("ul", class_="breadcrumb")
            genre = breadcrumb.find_all("li")[2].get_text().strip() if breadcrumb else "Not Available"
            
            # Description
            description_tag = book_soup.find("div", id="product_description")
            description =  description_tag.get_text(strip=True)[:100] if description_tag else "No description available"
            
            # Filter based on rating and price
            if price <= max_price and rating >= min_rating:
                all_books.append({
                    'UPC': upc,
                    'Title': title,
                    'Price (£)': price,
                    'Rating': rating,
                    'Genre': genre,
                    'Availability': availability,
                    'Description': description,
                    'Link': link
                })
        
        page_number += 1  # Move to next page
    
    # Convert to DataFrame
    df = pd.DataFrame(all_books)
    
    # Maximize the space for 'Link' and 'Description' columns
    pd.set_option('display.max_colwidth', None)
    
    return df


In [50]:
df_books = scrape_books(min_rating=3, max_price=20, num_books=50)
print(df_books)

              UPC                                    Title  Price (£)  Rating  \
0   Not Available                  The Coming Woman: A ...      17.93       3   
1   Not Available                              Set Me Free      17.46       5   
2   Not Available               The Four Agreements: A ...      17.66       5   
3   Not Available                           Sophie's World      15.94       5   
4   Not Available   Untitled Collection: Sabbath Poems ...      14.27       4   
5   Not Available                           Unicorn Tracks      18.78       3   
6   Not Available                          This One Summer      19.49       4   
7   Not Available                                   Thirst      17.27       5   
8   Not Available           The Life-Changing Magic of ...      16.77       3   
9   Not Available   Princess Jellyfish 2-in-1 Omnibus, ...      13.61       5   
10  Not Available  Princess Between Worlds (Wide-Awake ...      13.34       5   
11  Not Available           

In [51]:
df_books

,UPC,Title,Price (£),Rating,Genre,Availability,Description,Link
0,Not Available,The Coming Woman: A ...,17.93,3,Not Available,In stock,No description available,https://books.toscrape.com/catalogueng-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html
1,Not Available,Set Me Free,17.46,5,Not Available,In stock,No description available,https://books.toscrape.com/catalogueree_988/index.html
2,Not Available,The Four Agreements: A ...,17.66,5,Not Available,In stock,No description available,https://books.toscrape.com/catalogue-agreements-a-practical-guide-to-personal-freedom_970/index.html
3,Not Available,Sophie's World,15.94,5,Not Available,In stock,No description available,https://books.toscrape.com/catalogueworld_966/index.html
4,Not Available,Untitled Collection: Sabbath Poems ...,14.27,4,Not Available,In stock,No description available,https://books.toscrape.com/catalogue-collection-sabbath-poems-2014_953/index.html
5,Not Available,Unicorn Tracks,18.78,3,Not Available,In stock,No description available,https://books.toscrape.com/cataloguetracks_951/index.html
6,Not Available,This One Summer,19.49,4,Not Available,In stock,No description available,https://books.toscrape.com/catalogue-summer_947/index.html
7,Not Available,Thirst,17.27,5,Not Available,In stock,No description available,https://books.toscrape.com/catalogue46/index.html
8,Not Available,The Life-Changing Magic of ...,16.77,3,Not Available,In stock,No description available,https://books.toscrape.com/catalogue-changing-magic-of-tidying-up-the-japanese-art-of-decluttering-and-organizing_936/index.html
9,Not Available,"Princess Jellyfish 2-in-1 Omnibus, ...",13.61,5,Not Available,In stock,No description available,https://books.toscrape.com/catalogue-jellyfish-2-in-1-omnibus-vol-01-princess-jellyfish-2-in-1-omnibus-1_920/index.html
